In [2]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
from __future__ import division
import pickle
import random

### Get list of movies

Find 300 top grossing movies per year for 2004 - 2014

In [15]:
url_base = 'http://www.imdb.com/search/title?at=0&sort=boxoffice_gross_us&start=' 
url_feat = '&title_type=feature&year='

In [151]:
titles = {}
counts = [i for i in range(1,300,50)]

In [152]:
year = 2000

for count in counts:
    start = count
    url = url_base + str(start) + url_feat + str(year)
    print "Getting data " + str(start) + " for " + str(year)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page)

    lst = soup.find_all(attrs={"href" : re.compile("/title/")})
    for link in lst:
        if len(link.text) > 0 and link.text != 'X':
            titles[link.attrs['href']] = link.text

Getting data 1 for 2000
Getting data 51 for 2000
Getting data 101 for 2000
Getting data 151 for 2000
Getting data 201 for 2000
Getting data 251 for 2000


In [153]:
sorted(titles.values())

[u'102 Dalmatians',
 u'28 Days',
 u'3 Strikes',
 u'A Time for Drunken Horses',
 u'Aberdeen',
 u'About Adam',
 u'All the Pretty Horses',
 u'Almost Famous',
 u'American Psycho',
 u'Amores Perros',
 u'An Everlasting Piece',
 u'Autumn in New York',
 u'Bait',
 u'Bamboozled',
 u'Battlefield Earth',
 u'Beautiful',
 u'Beautiful Creatures',
 u'Bedazzled',
 u'Before Night Falls',
 u'Best in Show',
 u'Better Than Sex',
 u'Big Brother Trouble',
 u'Big Eden',
 u"Big Momma's House",
 u'Billy Elliot',
 u'Bless the Child',
 u'Blue Moon',
 u'Boiler Room',
 u'Bonhoeffer: Agent of Grace',
 u'Book of Shadows: Blair Witch 2',
 u"Boricua's Bond",
 u'Borstal Boy',
 u'Bossa Nova',
 u'Bounce',
 u'Boys Life 3',
 u'Boys and Girls',
 u'Bread and Roses',
 u'Bread and Tulips',
 u'Bring It On',
 u'Brother',
 u'Burnt Money',
 u'Call',
 u'Cast Away',
 u'Cecil B. DeMented',
 u'Center Stage',
 u'Chal Mere Bhai',
 u"Charlie's Angels",
 u'Chicken Run',
 u'Chocolat',
 u'Chopper',
 u'Chuck & Buck',
 u'Chunhyang',
 u'Company

In [154]:
len(titles.items())

300

In [155]:
len(movies_dct.items())

4116

In [21]:
# movies_dct = {}

In [26]:
def add_movie_to_data(soup2):
    title = soup2.find(itemprop = "name").text
    metacritic = str(soup2.find(attrs={"href" : re.compile("criticreviews?")}).text.strip())
    info = soup2.find(class_ = 'infobar')
    meta = info.find_all('meta')
    genres = []
    genres2 = info.find_all(itemprop='genre')
    for genre in genres2:
        genres.append(str(genre.text))
    
    duration = info.find(itemprop='duration').attrs['datetime']
#     budget = soup2.find(text=re.compile("Budget:")).next.strip()
    gross = soup2.find(text=re.compile("Gross:")).next.strip()
    opening = soup2.find(text=re.compile("Opening Weekend:")).next.strip()
    studio = soup2.find(attrs={"href" : re.compile("/company/")}).text.strip()
    try:
        rating = info.find(itemprop='contentRating')['content']
    except TypeError:
        rating = 'Unknown'
    
    try:
        release = meta[1].attrs['content']
    except IndexError:
        release = soup2.find(text=re.compile("Release Date:")).next.strip()
    
    movies_dct[title] = {'title': title,
                         'metacritic': metacritic,
                         'rating': rating,
                         'genres': genres,
                         'release': release,
                         'duration': duration,
                         'studio': studio,
#                          'budget': budget,
                         'gross': gross,
                         'opening': opening
                        }
    if len(movies_dct.items()) % 50 == 0:
        with open('my_data_partial.pkl', 'w') as picklefile:
            pickle.dump(movies_dct.items(), picklefile)
            print "dumped " + str(len(movies_dct.items()))
    return movies_dct

In [156]:
for href, title in titles.items():
    url_page = 'http://www.imdb.com/' + href

    try:
        response2 = requests.get(url_page)
        page2 = response2.text
        soup2 = BeautifulSoup(page2)
        add_movie_to_data(soup2)
        sleep(0.8)

    except AttributeError:
        print title + ' did not load'
        sleep(0.7)

dumped 4150
Raju Chacha did not load
Chal Mere Bhai did not load
dumped 4200
Refugee did not load
dumped 4250
One Week did not load
Phir Bhi Dil Hai Hindustani did not load
Khauff did not load
dumped 4300
dumped 4350
dumped 4350
Kahin Pyaar Na Ho Jaaye did not load
Hadh Kar Di Aapne did not load
Mela did not load
dumped 4400


In [157]:
len(movies_dct.items())

4406

In [158]:
(len(movies_dct.items())-4116)/len(titles.items())

0.9666666666666667

In [40]:
# movies_dct.items()[:20]

In [159]:
with open('my_data_2000_2014.pkl', 'w') as picklefile:
    pickle.dump(movies_dct.items(), picklefile)

In [168]:
count = 0
for i in range(len(movies_dct.items())):
    if (movies_dct.items()[i][1]['metacritic'] == 'Metacritic Reviews'):
        count += 1
        
print count

316


In [169]:
count / len(movies_dct.items())

0.07172038129822969

### MISC JUNK

In [9]:
url_page = 'http://www.imdb.com/title/tt3890264/'

response2 = requests.get(url_page)
page2 = response2.text
soup2 = BeautifulSoup(page2)

In [ ]:
#soup2.prettify()

In [10]:
soup2.find(itemprop = "name").text

u'Xin hua lu fang'

In [11]:
str(soup2.find(attrs={"href" : re.compile("criticreviews?")}).text.strip())

'Metacritic Reviews'

In [12]:
info = soup2.find(class_ = 'infobar')

In [13]:
len(info)

15

In [14]:
info.find(itemprop='contentRating')['content']

TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
meta = info.find_all('meta')

In [ ]:
meta

In [ ]:
info.find(itemprop='duration').attrs['datetime']

In [ ]:
str(soup2.find(text=re.compile("Opening Weekend:")).next.strip())

In [ ]:
str(soup2.find(attrs={"href" : re.compile("/company/")}).text.strip())

In [ ]:
str(soup2.find(text=re.compile("Release Date:")).next.strip())

In [ ]:
add_movie_to_data(soup2)
movies_dct